## Reliability Statistics Notebook

This Jupyter notebook shows how to compute reliability engineering 
statistics (reliability, confidence, and assurance) using the
[relistats library](https://github.com/sanjaymjoshi/relistats).

This notebook was used to generate the plots and tables in the paper:

S.M. Joshi, "Computation of Reliability Statistics for Success-Failure Experiments,"
[arXiv:2303.03167](https://doi.org/10.48550/arXiv.2303.03167), March 2023.
  

In [ ]:
# For online mode, such as google colab or GitHub codespace
# You can use `pipenv install` for local use and not execute this cell
!pip install relistats
!pip install tabulate


## Set up
Run the follow cell first to import packages needed.

In [ ]:

from relistats.binomial import assurance, confidence, reliability
import matplotlib.pyplot as plt
from tabulate import tabulate

#table_format = "latex" # for paper
table_format = "simple" # for notebook


## Reliability

Reliability is probability of success. For success-failure experiments or test units
that either pass or fail, the samples typically have 'binomial distribution'. They are
assumed to be independent (one sample passing or failing has no effect on another sample)
and identically distributed (the reliability is same for each sample).

The math assumes infinite number of samples, but we can get access to only a finite
number of samples. By observing the number of successes and failures, we can compute an
estimate of reliability. Based on the number of samples, we qualify the quality of this
estimate using 'confidence'. Then we can say something linke "we have 95% confidence that
the reliability is at least about 74% if we see zero failures in 10 samples" (see plot and
table generated by the block below).

In the block below, the number of samples vary from 10 to 100 in steps of 10 and the number
of failures vary from 0 to 5. The confidence is held constatnt at 95%. If you are using this
notebook on Google Colab, you can change these numbers on the right and see how the plot
and table changes.

In [ ]:
#@title Reliability computations { vertical-output: true }
n_start = 10 #@param {type:"integer"}
n_end = 100 #@param {type:"integer"}
n_step = 10 #@param {type:"integer"}
f_start = 0 #@param {type:"integer"}
f_end = 5 #@param {type:"integer"}
f_step = 1 #@param {type:"integer"}
c = 95 #@param {type:"integer"}

N = range(n_start, n_end+n_step, n_step) # number of samples
F = range(f_start, f_end+f_step, f_step) # number of failures
all_a = []
for f in F:
  aa =  [100 *reliability(n, f, c/100) for n in N]
  plt.plot(N, aa)
  all_a.append(aa)

plt.ylabel(f'Reliability (%) at {c}% confidence')
plt.xlabel('Total # samples (including failures)')
plt.legend(F)
plt.grid()
plt.show()

print(tabulate(all_a, headers=N, tablefmt=table_format, floatfmt=".1f", showindex="always"))

## Confidence

Confidence in reliability is probability that the actual reliability of the whole,
infinite, population is at least the estimated reliability. 

As the number of samples increases, the confidence in reliability value also increases.
For the same number of samples, the confidence in minimum reliability increases as the
reliability level drops. You can see this behaviour in the plot and table below.

In the block below, the number of samples vary from 10 to 100 in steps of 10 and the
number of failures vary from 0 to 5. The minimum reliability level is held constatnt
at 90%. If you are using this notebook on Google Colab, you can change these numbers on
the right and see how the plot and table changes.

In [ ]:
#@title Confidence calculations { vertical-output: true }
n_start = 10 #@param {type:"integer"}
n_end = 100 #@param {type:"integer"}
n_step = 10 #@param {type:"integer"}
f_start = 0 #@param {type:"integer"}
f_end = 5 #@param {type:"integer"}
f_step = 1 #@param {type:"integer"}
reli_percent = 90 #@param {type:"integer"}

N = range(n_start, n_end+n_step, n_step) # number of samples
F = range(f_start, f_end+f_step, f_step) # number of failures
r = reli_percent/100 # reliability
all_c = []
for f in F:
  cc =  [100 *confidence(n, f, r) for n in N]
  plt.plot(N, cc)
  all_c.append(cc)

plt.ylabel(f'Confidence (%) at {reli_percent}% minimum reliability')
plt.xlabel('Total # samples (including failures)')
#plt.title('Confidence Level for Number of Failures')
plt.legend(F)
plt.grid()
plt.show()

print(tabulate(all_c, headers=N, tablefmt=table_format, floatfmt=".1f", showindex="always"))

# Trade-off

There is a constant trade-off between reliability and confidence. Higher reliability means lower confidence and vice versa. The trade-off is optimal at the cross-over point between these two as shown in the code plot below. The shaded area shows the minimum of reliability and confidence. This minimum is highest at the cross-over point, known as assurance.

In [ ]:
#@title Trade-off { vertical-output: true }
n_start = 10 #@param {type:"integer"}
n_end = 10 #@param {type:"integer"}
n_step = 5 #@param {type:"integer"}

f_start = 0 #@param {type:"integer"}
f_end = 5 #@param {type:"integer"}
f_step = 1 #@param {type:"integer"}
show_assurance = True #@param {type:"boolean"}

if show_assurance:
    all_r = range(100)
    plt.plot(all_r, all_r, label='reliability')

N = range(n_start, n_end+n_step, n_step) # number of samples
F = range(f_start, f_end+f_step, f_step) # number of failures

for n in N:
    for f in F:
        all_c = [confidence(n, f, r/100)*100 for r in all_r]
        plt.plot(all_r, all_c, label=f"n={n}, f={f}")
        if show_assurance:
            plt.fill_between(all_r, [min(r,c) for r,c in zip(all_r, all_c)], color='cyan', alpha=0.5)
plt.grid()
plt.legend()
plt.xlabel('reliability')
plt.ylabel(f'confidence at {f} failures')
plt.show()

## Assurance

Assurance simplifies reliability and confidence by setting both of them the same.
The result is just one number that is easier to communicate. For example, 90% assurance
means 90% reliability with 90% confidence. Given the number of samples and number of
failures, assurance is just one number.

As the number of samples increases, the assurance improves. You can see this behavior
in the plot and table below.

In the block below, the number of samples vary from 10 to 100 in steps of 10 and the
number of failures vary from 0 to 5. If you are using this notebook on Google Colab,
you can change these numbers on the right and see how the plot and table changes.

In [ ]:
#@title Assurance computations { vertical-output: true }
n_start = 10 #@param {type:"integer"}
n_end = 100 #@param {type:"integer"}
n_step = 10 #@param {type:"integer"}
f_start = 0 #@param {type:"integer"}
f_end = 5 #@param {type:"integer"}
f_step = 1 #@param {type:"integer"}

N = range(n_start, n_end+n_step, n_step) # number of samples
F = range(f_start, f_end+f_step, f_step) # number of failures
all_a = []
for f in F:
  aa =  [100 *assurance(n, f) for n in N]
  plt.plot(N, aa)
  all_a.append(aa)

plt.ylabel(f'Assurance (%)')
plt.xlabel('Total # samples (including failures)')
#plt.title('Confidence Level for Number of Failures')
plt.legend(F)
plt.grid()
plt.show()

print(tabulate(all_a, headers=N, tablefmt=table_format, floatfmt=".1f", showindex="always"))

## Accuracy of Reliability Approximation

This section was created to assess the quality of reliability approximations using three methods:
- Wilson Score Interval
- Wilson Score Interval with Continuity Correction
- Numerical optimization
See the paper referenced above for details.

The numerical optimization is accurate and fast technique to calculate reliability or assurance.

In [ ]:
#@title Accuracy of Closed Form Reliability Computations { vertical-output: true }
n = 40 #@param {type:"integer"}

from relistats.binomial import _wilson_lower, _wilson_lower_corrected, reliability_optim


rmat = []
all_ns = range(1,n)
for ns in all_ns:
  r = ns/n
  c = confidence(n, n-ns, r)
  rw = _wilson_lower(r, n, c)
  rwc = _wilson_lower_corrected(r, n, c)
  roptim = reliability_optim(n, n-ns, c)

  rmat.append([(rw-r)*100, (rwc-r)*100, (roptim-r)*100 ])

plt.plot(all_ns, rmat)
  
plt.legend(['Wilson', 'Wilson corrected', 'Numerical method'])
plt.xlabel(f'Good samples, out of {n}')
plt.ylabel('Reliability estimate difference (%)')
#plt.title('Accuracy of Wilson Score Interval Methods')
plt.grid()
plt.show()

